In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row
from pyspark.sql.functions import lit

In [2]:
# load up movie ID -> movie name dictionary
def loadMovieNames():
  movieNames = {}
  with open("/dbfs/FileStore/tables/u.item", encoding = "ISO-8859-1") as f:
    for line in f:
      fields = line.split('|')
      movieNames[int(fields[0])] = fields[1]
  return movieNames
# Concert u.data lines into (userID, movieID, rating) rows
def parseInput(line):
  fields = line.value.split()
  return Row(userID = int(fields[0]), movieID = int(fields[1]), rating = float(fields[2]))
  
if __name__ == '__main__':
#   Create a SparkSession (the config bit is only for Windows)
  spark = SparkSession.builder.appName('Movie_Recommender').getOrCreate()
  spark.conf.set("spark.sql.crossJoin.enabled", "true")
#   load up our movieID -> name dictionary
  movieNames = loadMovieNames()
#   Get the raw data
  lines = spark.read.text("/FileStore/tables/u.data").rdd
#   Cinvert it to a RDD of Row objects with (userID, movieID, rating)
  ratingsRDD = lines.map(parseInput)
#   Convert it to a DataFrame and cache it
  ratings = spark.createDataFrame(ratingsRDD).cache()
#   Create an ALS collaborative filtering model from the complete data set
  als = ALS(maxIter=5, regParam=0.01, userCol="userID", itemCol="movieID", ratingCol="rating")
  model = als.fit(ratings)
  
#   Print out ratings from user 196:
  print("\nRatings for userID 196:")
  userRatings = ratings.filter("userID = 196")
  for rating in userRatings.collect():
    print(movieNames[rating['movieID']], rating['rating'])
  
  print("\nTop 20 recommendations:")
#   Find movies rated more than 100 times
  ratingCounts = ratings.groupBy("movieID").count().filter("count > 100")
#   Construct a "test" dataframe for user 196 with every movie rated more than 100 times
  popularMovies = ratingCounts.select("movieID").withColumn('userID', lit(196))
  
#   Run our model on that list of popular movies for user ID 196
  recommendations = model.transform(popularMovies)
#   Get the top 20 movies with the highest predicted rating for this user
  topRecommendations = recommendations.sort(recommendations.prediction.desc()).take(20)

  for recommendation in topRecommendations:
    print(movieNames[recommendation['movieID']], recommendation['prediction'])
    
#   spark.stop()
  

Ratings for userID 196:
Kolya (1996) 3.0
Mrs. Doubtfire (1993) 4.0
Muriel's Wedding (1994) 4.0
Shall We Dance? (1996) 3.0
Stand by Me (1986) 5.0
Ace Ventura: Pet Detective (1994) 5.0
Mrs. Brown (Her Majesty, Mrs. Brown) (1997) 4.0
Raising Arizona (1987) 4.0
Being There (1979) 5.0
Truth About Cats & Dogs, The (1996) 4.0
Englishman Who Went Up a Hill, But Came Down a Mountain, The (1995) 2.0
Birdcage, The (1996) 4.0
English Patient, The (1996) 5.0
Home Alone (1990) 3.0
American President, The (1995) 5.0
Babe (1995) 5.0
Harold and Maude (1971) 4.0
Up in Smoke (1978) 4.0
Four Weddings and a Funeral (1994) 3.0
While You Were Sleeping (1995) 3.0
Men in Black (1997) 2.0
Kids in the Hall: Brain Candy (1996) 4.0
Groundhog Day (1993) 3.0
Boogie Nights (1997) 3.0
Marvin's Room (1996) 3.0
Cold Comfort Farm (1995) 3.0
Adventures of Priscilla, Queen of the Desert, The (1994) 4.0
Secrets & Lies (1996) 5.0
Van, The (1996) 3.0
Waiting for Guffman (1996) 4.0
Nutty Professor, The (1996) 4.0
Fish Called Wanda, A (1988) 5.0
Mighty Aphrodite (1995) 2.0
Beautiful Girls (1996) 3.0
Princess Bride, The (1987) 2.0
Fast, Cheap & Out of Control (1997) 4.0
That Thing You Do! (1996) 4.0
Full Monty, The (1997) 3.0
Operation Dumbo Drop (1995) 1.0

Top 20 recommendations:
Manchurian Candidate, The (1962) 5.7640604972839355
Chinatown (1974) 5.721989631652832
Miracle on 34th Street (1994) 5.604461193084717
Good, The Bad and The Ugly, The (1966) 5.577101230621338
To Kill a Mockingbird (1962) 5.560796737670898
Apocalypse Now (1979) 5.504014492034912
One Flew Over the Cuckoo's Nest (1975) 5.49299430847168
North by Northwest (1959) 5.465051174163818
Amadeus (1984) 5.462077617645264
Some Like It Hot (1959) 5.453675270080566
Carrie (1976) 5.444283485412598
Pinocchio (1940) 5.412817001342773
Raging Bull (1980) 5.342199325561523
Psycho (1960) 5.31464147567749
Rear Window (1954) 5.2964324951171875
Sling Blade (1996) 5.2784624099731445
Dumbo (1941) 5.259562969207764
Taxi Driver (1976) 5.253085613250732
Glory (1989) 5.248711585998535
Graduate, The (1967) 5.240118503570557

In [3]:
spark.stop()